## Monte Carlo Simulation ##

### Calculating pi ###

In [3]:
import random
import numpy as np

def estimate_pi(num_points):
    points_inside_circle = 0

    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)

        distance = x**2 + y**2
        if distance <= 1:
            points_inside_circle += 1

    return 4 * points_inside_circle / num_points

# Repeat the simulation 5 times for each number of points and compute the mean
num_points_list = [1000, 10000, 100000, 1000000]
num_simulations = 5

for num_points in num_points_list:
    pi_estimates = []
    for _ in range(num_simulations):
        estimated_pi = estimate_pi(num_points)
        pi_estimates.append(estimated_pi)
    
    mean_pi = np.mean(pi_estimates)
    print(f"Number of points: {num_points}, Mean Estimated Pi: {mean_pi}")


Number of points: 1000, Mean Estimated Pi: 3.1184000000000003
Number of points: 10000, Mean Estimated Pi: 3.14272
Number of points: 100000, Mean Estimated Pi: 3.1382879999999997
Number of points: 1000000, Mean Estimated Pi: 3.1425736


### Mensch ###



In [ ]:
import random

def play_game():
    positions = [-1, -1, -1, -1]  # Positions of players' pieces
    out_areas = [1, 1, 1, 1]  # Out areas for each player
    start_fields = [1, 11, 21, 31]  # Start fields for each player
    players = [0, 1, 2, 3]  # Player IDs

    def move_piece(player, steps):
        current_position = positions[player]
        if can_move(player, steps):
            new_position = (current_position + steps) % 40
            positions[player] = new_position
    
    def is_winner(player, steps):
        return (positions[player] + steps)%40 == start_fields[player] 
    
    def check_collision(player):
        current_position = positions[player]
        for other_player in players:
            if other_player != player and positions[other_player] == current_position:
                positions[other_player] = -1
                out_areas[other_player] = 1
    
    def can_move(player, steps):
        if (positions[player] + steps)%40 > start_fields[player] :
            return False
        return True

    while True:
        for player in players:
            dice_roll = random.randint(1, 6)

            if out_areas[player] == 1 and dice_roll != 6:
                continue

            if out_areas[player] == 1:
                positions[player] = start_fields[player]
                out_areas[player] = 0
                check_collision(player)
                continue

            if not is_winner(player, dice_roll):
                move_piece(player, dice_roll)
                continue

            if is_winner(player, dice_roll):
                print("some one won!")
                return player

# Monte Carlo simulation
num_simulations = 10000
win_counts = [0, 0, 0, 0]

for _ in range(num_simulations):
    winner = play_game()
    win_counts[winner] += 1

total_games = sum(win_counts)
probabilities = [count / total_games for count in win_counts]

for player, prob in enumerate(probabilities):
    print(f"Player {player + 1}: Probability of winning - {prob:.4f}")


## Central Limit Theorem(CLT) ##

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the random seed for reproducibility
np.random.seed(42)

# Function to generate samples, calculate means, and plot histograms
def clt_experiment(distribution, sample_sizes):
    for sample_size in sample_sizes:
        sample_means = []
        for _ in range(1000):  # Generate 1000 samples
            sample = distribution(size=sample_size)
            sample_mean = np.mean(sample)
            sample_means.append(sample_mean)
        
        # Plot histogram of sample means
        plt.figure(figsize=(8, 6))
        sns.histplot(sample_means, kde=True, stat='density', color='skyblue', linewidth=0)
        
        # Overlay with expected normal distribution
        mean = np.mean(sample_means)
        std = np.std(sample_means)
        x = np.linspace(mean - 4*std, mean + 4*std, 1000)
        y = 1/(std * np.sqrt(2 * np.pi)) * np.exp(-(x - mean)**2 / (2 * std**2))
        plt.plot(x, y, color='red', linestyle='--', linewidth=2)
        
        plt.title(f'Sample Size: {sample_size}')
        plt.xlabel('Sample Mean')
        plt.ylabel('Density')
        plt.legend(['Expected Normal Distribution', 'Sample Means'])
        plt.show()

# Normal Distribution
normal_distribution = lambda size: np.random.normal(loc=0, scale=1, size=size)
sample_sizes = [10, 30, 100]
clt_experiment(normal_distribution, sample_sizes)

# Exponential Distribution
exponential_distribution = lambda size: np.random.exponential(scale=1, size=size)
clt_experiment(exponential_distribution, sample_sizes)

# Uniform Distribution
uniform_distribution = lambda size: np.random.uniform(low=0, high=1, size=size)
clt_experiment(uniform_distribution, sample_sizes)


## Hypothesis Testing ##



### Unfair Coin ###

In [ ]:
import numpy as np
import scipy.stats as stats

# Simulate unfair coin toss
np.random.seed(42)

def simulate_coin_toss(prob_head, num_tosses):
    return np.random.choice(['H', 'T'], size=num_tosses, p=[prob_head, 1 - prob_head])

# Hypothesis testing function
def hypothesis_test(data, null_hypothesis):
    z_score, p_value = stats.binom_test(sum(data == 'H'), n=len(data), p=null_hypothesis, alternative='two-sided')
    return z_score, p_value

# Perform hypothesis testing for different sample sizes
sample_sizes = [30, 100, 1000]
prob_head = 0.55  # Probability of landing on one face
null_hypothesis = 0.5  # Null hypothesis: The coin is fair

for size in sample_sizes:
    coin_tosses = simulate_coin_toss(prob_head, size)
    z_score, p_value = hypothesis_test(coin_tosses, null_hypothesis)
    
    confidence_interval = stats.norm.interval(0.95, loc=np.mean(coin_tosses == 'H'), scale=np.sqrt(null_hypothesis*(1-null_hypothesis)/size))
    
    print(f"Sample size: {size}")
    print(f"Z-score: {z_score}")
    print(f"P-value: {p_value}")
    print(f"Confidence Interval: {confidence_interval}")
    
    if p_value < 0.05:
        print("Reject null hypothesis: The coin is biased.")
    else:
        print("Fail to reject null hypothesis: The coin is fair.")
    
    print("\n")


### Job Placement ###


In [ ]:
import pandas as pd
from scipy import stats

# Read data from CSV file
data = pd.read_csv("job_placement.csv")

# Split students into two groups based on job placement status
working_students = data[data['placement_status'] == 'Working']
non_working_students = data[data['placement_status'] == 'Not Working']

# Calculate mean grades for working and non-working students
working_grades = working_students['gpa']
non_working_grades = non_working_students['gpa']

# Calculate sample statistics
n1 = len(working_grades)
n2 = len(non_working_grades)
mean1 = working_grades.mean()
mean2 = non_working_grades.mean()
var1 = working_grades.var(ddof=1)
var2 = non_working_grades.var(ddof=1)

# Calculate pooled variance
sp_squared = ((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2)

# Calculate t-statistic
t_statistic = (mean1 - mean2) / (sp_squared * (1/n1 + 1/n2))**0.5

# Calculate degrees of freedom
df = n1 + n2 - 2

# Calculate p-value using t-distribution
p_value = 2 * (1 - stats.t.cdf(abs(t_statistic), df=df))

# Report the results
print(f"t-statistic: {t_statistic}")
print(f"Degrees of freedom: {df}")
print(f"P-value: {p_value}")

# Make decision based on significance level (α = 0.05)
alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis: Working alongside studying has a significant impact on grades.")
else:
    print("Fail to reject null hypothesis: There is no significant impact of working alongside studying on grades.")
